In [1]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [2]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.13.3
pandas version:   0.20.3
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML file. (Actually, there are several runs that have been concatinated into a YAML file that has a sequence in its top level.) Most of these runs were performed with the miniGraphics application compiled at SHA 50630f94848b970cc8894eb0ceb84b47bcef20f5. The execption are the 234 composite runs (which was implemented afterward). These are compiled at SHA 4e647c2bb7fe5b77a1d5340c3480b146dcbf3947.

In [3]:
filename = 'miniGraphics-skybridge-vn-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.DataFrame(yaml_data)

Some of the original runs gave suspect results where the runs took much longer than expected. These appear to be transient issues in the system since a repeat of the run is closer to what is expected. To get more realistic results, I re-ran these conditions. So jettison these runs from the original data and load up the re-runs.

In [4]:
# Remove incorrect readings
data = data.loc[
    ((data['composite-algorithm'] != '2-3-SwapBase') |
     (data['num-processes'] != 4096) |
     (data['image-height'] != 1080)) &
    ((data['composite-algorithm'] != 'BinarySwapFold') |
     (data['num-processes'] != 2896) |
     (data['image-height'] != 1080)) &
    ((data['composite-algorithm'] != 'BinarySwapRemainder') |
     (data['num-processes'] != 2048) |
     (data['image-height'] != 1080)) &
    ((data['composite-algorithm'] != 'BinarySwapTelescoping') |
     (data['num-processes'] != 1616) |
     (data['image-height'] != 1080))
]
# Load correct readings and add them to the data
filename = 'miniGraphics-skybridge-vn-corrections.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.concat([data, pandas.DataFrame(yaml_data)], ignore_index=True)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [5]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [6]:
data = flatten_table(data)

Add a column that gives a human-readable name to each image resolution.

In [7]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data['image-size'] = data['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [8]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'BinarySwap234Schedule': '234 Composite',
    'IceTBase': 'IceT'
}

data['composite-algorithm'] = data['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [9]:
import IPython.display

data_description = ''

for column_name in data.columns:
    data_description = data_description + '**' + column_name + '**: '
    unique_values = data[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    elif (numpy.issubdtype(unique_values.dtype, numpy.number)):
        data_description = (
            data_description +
            str(numpy.nanmin(unique_values)) + ' &ndash; ' +
            str(numpy.nanmax(unique_values)) + ' '
        )
    elif not pandas.isnull(unique_values).any():
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

**color-buffer-format**: byte  
**composite-algorithm**: Naive Remainder Telescoping 2-3 Swap IceT 234 Composite  
**composite-seconds**: 0.00119348 &ndash; 3.49925  
**compress-seconds**:  
**construct-tree-seconds**: 0.000182052 &ndash; 0.0610641  
**depth-buffer-format**: float  
**gather-seconds**: 0.00071677 &ndash; 3.45673  
**geometry**: box  
**geometry-distribution**: duplicate  
**geometry-overlap**: -0.05  
**icet-copy-result-seconds**:  
**image-compression**: True False  
**image-height**: 288 1080 4320  
**image-width**: 512 1920 7680  
**num-processes**: 128 &ndash; 8192  
**num-triangles**: 1536 &ndash; 98304  
**paint-seconds**: 0.00011823 &ndash; 0.147793  
**painter**: simple  
**partial-composite-seconds**: 0.000293153 &ndash; 0.86969  
**phi-rotation**: -178.406 &ndash; 136.756  
**random-seed**: 17627  
**rendering-order-dependent**: False  
**start-time**: 2018-05-30T02:23:10.000000000 &ndash; 2018-06-08T12:10:45.000000000  
**theta-rotation**: -177.945 &ndash; 179.418  
**total-seconds**: 0.00146716 &ndash; 3.59942  
**trial-num**: 0 &ndash; 19  
**uncompress-seconds**:  
**zoom**: 1.5  
**image-size**: nan HDTV 8K UHD  


## Plot Data

We are plotting the time it takes to do a "partial composite" (that is the time to blend all the pixels, but the pixels are left distributed across all the processes).

The first thing we want to do is to average the time it took over all trials. This is easily done with a pivot table.

In [10]:
average_partial_composite = data.pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

print(average_partial_composite.index)
print(average_partial_composite.columns)

Int64Index([ 128,  160,  176,  192,  224,  256,  272,  320,  352,  400,  448,
             512,  560,  640,  720,  800,  912, 1024, 1136, 1280, 1440, 1616,
            1824, 2048, 2288, 2576, 2896, 3248, 3648, 4096, 4592, 5152, 5792,
            6496, 7296, 8192],
           dtype='int64', name=u'num-processes')
MultiIndex(levels=[[u'8K UHD', u'HDTV'], [u'2-3 Swap', u'234 Composite', u'IceT', u'Naive', u'Remainder', u'Telescoping']],
           labels=[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], [0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]],
           names=[u'image-size', u'composite-algorithm'])


Make a grouping structure of the data so we can pull out the actual data values for each trial.

In [11]:
grouped_data = data.groupby(['image-size', 'composite-algorithm'])

Plot data.

In [12]:
image_size = 'HDTV'
algorithms = ['2-3 Swap', 'Naive', 'Remainder', 'Telescoping', '234 Composite', 'IceT']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    # Some adjustments for correct label placement
    if algorithm == 'Naive':
        vert_shift = '70%'
    elif algorithm == '234 Composite':
        vert_shift = '150%'
    elif algorithm == 'Telescoping':
        vert_shift = '-20%'
    elif algorithm == 'Remainder':
        vert_shift = '-100%'
    else:
        vert_shift = '0'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm + '  ',
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'2px',
            'baseline-shift': vert_shift,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 45.0 205.22496311664361 L 61.093195036721156 203.97416744545376 L 67.967000277976638 204.79975066352659 L 74.242292806309777 204.8001118961285 L 85.359696562289386 203.99997473614692 L 94.990029737391978 205.64263831258646 L 99.362299772413223 204.79394315477305 L 111.08322477411312 205.24819315165789 L 117.9570300153686 205.41180373319378 L 127.17641981083435 204.88349410613932 L 135.34972629968135 204.70410738539371 L 144.98005947478396 204.92485523905651 L 151.44292133640249 205.29304156853948 L 161.0732545115051 205.51915928376511 L 169.56781038673265 205.01511476206497 L 177.1664495482263 204.30409090625895 L 186.61625605085612 204.46566608101878 L 194.97008921217591 202.99847124095047 L 202.45595216930371 203.04828660610815 L 211.06328424889705 197.14454409803997 L 219.55784012412465 201.81757407822761 L 227.87410080843091 196.25960758476322 L 236.60628578824813 199.66403034777517 L 244.96011894956789 197.4939880129092 L 252.95209182927613 196.3105830623154 L 261.50266352712248 194.58345357805936 L 269.94742970897028 191.22333043624539 L 278.22028123500968 187.78205500105912 L 286.59631552564008 184.26482346454341 L 294.95014868695978 176.98793858173528 L 303.19385085537931 172.39364341519945 L 311.49269326451446 165.82963717867241 L 319.93745944636231 157.3848456132481 L 328.21031097240171 146.456698005059 L 336.586345263032 134.06902480281116 L 344.9401784243517 93.327773977296772" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap <path d="M 45.0 206.83542836404868 L 61.093195036721156 185.67196338454636 L 67.967000277976638 185.2825060122363 L 74.242292806309777 185.97972661458138 L 85.359696562289386 185.89433678491773 L 94.990029737391978 207.27358266980676 L 99.362299772413223 186.78125313072408 L 111.08322477411312 186.68745074950465 L 117.9570300153686 187.36085778724782 L 127.17641981083435 187.19631633708397 L 135.34972629968135 187.28001115223108 L 144.98005947478396 207.90103675252016 L 151.44292133640249 187.59413763348411 L 161.0732545115051 187.78257600961956 L 169.56781038673265 187.95865606270172 L 177.1664495482263 187.76302082165154 L 186.61625605085612 188.09941173538681 L 194.97008921217591 208.23435024235854 L 202.45595216930371 188.21574252675913 L 211.06328424889705 188.14923404732531 L 219.55784012412465 188.19840336186809 L 227.87410080843091 187.85990410015825 L 236.60628578824813 187.99229932214371 L 244.96011894956789 207.55333300624275 L 252.95209182927613 187.70141676946659 L 261.50266352712248 187.52001548228725 L 269.94742970897028 187.07502554421643 L 278.22028123500968 186.74399754526308 L 286.59631552564008 185.54069701444183 L 294.95014868695978 205.15259850099062 L 303.19385085537931 185.47957923691325 L 311.49269326451446 184.64761192728 L 319.93745944636231 182.38225281297895 L 328.21031097240171 180.81776136094598 L 336.586345263032 178.2188734066016 L 344.9401784243517 201.0503993931691" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive <g class="toyplot-Datum" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:1.0;opacity:1.0;stroke:rg

In [13]:
toyplot.pdf.render(canvas, 'scaling-hdtv.pdf')

In [14]:
image_size = '8K UHD'
algorithms = ['2-3 Swap', 'Naive', 'Remainder', 'Telescoping', '234 Composite', 'IceT']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    if algorithm == 'Naive':
        vert_shift = '-30%'
    elif algorithm == '234 Composite':
        vert_shift = '50%'
    elif algorithm == 'Telescoping':
        vert_shift = '-160%'
    elif algorithm == 'Remainder':
        vert_shift = '-240%'
    else:
        vert_shift = '0'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm + '  ',
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'2px',
            'baseline-shift': vert_shift,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 45.0 132.72374416666668 L 61.093195036721156 128.84541488888891 L 67.967000277976638 127.99216733333334 L 74.242292806309777 129.44559461111112 L 85.359696562289386 129.55720611111113 L 94.990029737391978 135.40715272222221 L 99.362299772413223 131.83048050000002 L 111.08322477411312 132.34874488888889 L 117.9570300153686 132.15321105555557 L 127.17641981083435 132.5085243888889 L 135.34972629968135 133.46945616666667 L 144.98005947478396 137.02231694444447 L 151.44292133640249 135.02737133333338 L 161.0732545115051 136.14977988888893 L 169.56781038673265 133.85325155555554 L 177.1664495482263 133.63757338888891 L 186.61625605085612 136.63941355555556 L 194.97008921217591 139.40166499999998 L 202.45595216930371 137.6978335 L 211.06328424889705 137.42397711111113 L 219.55784012412465 136.48425511111117 L 227.87410080843091 133.78207738888889 L 236.60628578824813 133.32086383333336 L 244.96011894956789 139.73736672222222 L 252.95209182927613 138.67743533333336 L 261.50266352712248 137.52661905555559 L 269.94742970897028 136.35601772222222 L 278.22028123500968 134.22947733333336 L 286.59631552564008 136.56835088888892 L 294.95014868695978 132.37155283333334 L 303.19385085537931 133.34579027777778 L 311.49269326451446 133.92248066666664 L 319.93745944636231 131.24811600000001 L 328.21031097240171 126.498439 L 336.586345263032 129.33856700000001 L 344.9401784243517 127.34197366666669" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap <path d="M 45.0 126.39811377777779 L 61.093195036721156 68.120049500000022 L 67.967000277976638 68.028830111111148 L 74.242292806309777 69.634070833333382 L 85.359696562289386 70.019179444444433 L 94.990029737391978 138.25750155555556 L 99.362299772413223 73.159948611111105 L 111.08322477411312 72.422115944444442 L 117.9570300153686 73.651787500000026 L 127.17641981083435 74.011994444444497 L 135.34972629968135 73.830645888888881 L 144.98005947478396 139.9580747777778 L 151.44292133640249 75.717783388888904 L 161.0732545115051 77.176141444444468 L 169.56781038673265 77.775354833333353 L 177.1664495482263 77.939482833333358 L 186.61625605085612 77.492689944444464 L 194.97008921217591 142.95996450000004 L 202.45595216930371 80.256936000000053 L 211.06328424889705 70.017098111111125 L 219.55784012412465 81.354257055555578 L 227.87410080843091 81.157261333333338 L 236.60628578824813 81.059500611111147 L 244.96011894956789 144.56427605555558 L 252.95209182927613 82.966571833333333 L 261.50266352712248 82.987843555555628 L 269.94742970897028 77.496257944444466 L 278.22028123500968 83.28407427777779 L 286.59631552564008 83.279663833333373 L 294.95014868695978 140.4405352777778 L 303.19385085537931 84.232010111111137 L 311.49269326451446 83.822544944444459 L 319.93745944636231 83.865918444444503 L 328.21031097240171 83.248840277777788 L 336.586345263032 83.152293666666708 L 344.9401784243517 144.71965750000001" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive <g class="toyplot-Datum" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:1.0;opacity:1.0;stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0" transform="translate(286.59631

In [15]:
toyplot.pdf.render(canvas, 'scaling-8k.pdf')